In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [2]:
from load_functions import load_text
    
text_input = load_text(day=15)
print(len(text_input))
print(text_input[0])

100
7467923161199853975163878964955815881991642676938912451447919792683728694766149341499879987416299819


Convert to array

In [3]:
import numpy as np

list_input = []
for t in text_input:
    list_input.append([int(n) for n in list(t)])

arr_input = np.array(list_input)
arr_input

array([[7, 4, 6, ..., 8, 1, 9],
       [1, 7, 9, ..., 2, 3, 9],
       [9, 4, 8, ..., 6, 4, 9],
       ...,
       [6, 9, 8, ..., 1, 7, 9],
       [9, 9, 9, ..., 9, 1, 9],
       [2, 7, 9, ..., 7, 1, 9]])

# Problem 

#### Part 1: Find the shortest path from top left to bottom right

Taken from https://github.com/statneutrino/AdventOfCode/blob/main/2021/python/day15.py, struggled to get a solution that scaled well. Using a list to append to / remove from (nodes visited / to visit) and have to repeatedly scan these doesn't work on large datasets.

The below solution instead stores distances and nodes visited in arrays.

In [50]:
import copy
import numpy as np

# Get adjacent nodes
def get_adjacent_nodes(node, array):
    
    adjacent_nodes = []
    (max_height, max_width) = array.shape
    (h, w) = node
    for offset in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
        new_h = (h + offset[0])
        new_w = (w + offset[1])
        
        if new_h >= 0 and new_h <= max_height - 1 and new_w >= 0 and new_w <= max_width - 1:
            adjacent_nodes.append((new_h, new_w))
            
    return adjacent_nodes


# Using Dijkstra's algorithm
def find_shortest_path(array, start_node, dest_node):   
    
    # Initialise matrices for visited (boolean) and distances.
    visited = np.zeros(array.shape, dtype=bool) # Zeros evaluate to False
    distance_matrix = np.full(array.shape, 
                              np.inf, # Initialise distances as infinity
                              dtype=float) 
    distance_matrix[start_node] = 0
    current_node = start_node
    
    while visited[dest_node] == False:
        
        # For unvisited nodes
        if visited[current_node] == False:
            
            # Loop through adjacent arrays
            adjacent_nodes = get_adjacent_nodes(current_node, array)
            
            for adj in adjacent_nodes:
                dist_from_current_to_adj = distance_matrix[current_node] + array[adj]
                
                # If distance from current node to adjacent is shorter than the current distance stored, update it
                if dist_from_current_to_adj < distance_matrix[adj]:
                    distance_matrix[adj] = dist_from_current_to_adj
                    
            visited[current_node] = True
            
        # find minimum unvisited node
        # np.where, when only passed a condition, returns all elements of the array that meet that condition
        nodes_min_d = np.where( 
                            np.logical_and(
                                # First, check
                                distance_matrix == np.amin(distance_matrix[np.invert(visited)]),
                                # Second, check node hasn't been visited
                                np.invert(visited))
                        )
        current_node = (nodes_min_d[0][0], nodes_min_d[1][0])
        if current_node == dest_node:
            return int(distance_matrix[dest_node])

print('Day 15, part 1:', find_shortest_path(arr_input, (0,0), (arr_input.shape[0]-1,
                                                               arr_input.shape[1]-1)))

Day 15, part 1: 621


#### Part 2: the cave is actually 5x bigger

Need to repeat the array into a 5x5 grid of the original, with each move down or right adding 1 to the distance scores. 

Any score > 9 is reset to 1.

In [54]:
arr_input_pt2 = copy.deepcopy(arr_input)
init_rows, init_cols = arr_input_pt2.shape 

# Set up 5x1 grid
for row in range(4):
    arr_input_pt2 = np.concatenate((arr_input_pt2, np.where(arr_input_pt2[-init_rows:]+1 > 9,
                                                            1, 
                                                            arr_input_pt2[-init_rows:]+1)), axis=0)
    
# Set up 5x5 grid
for col in range(4):
    arr_input_pt2 = np.concatenate((arr_input_pt2, np.where(arr_input_pt2[:, -init_cols:]+1 > 9,
                                                            1, 
                                                            arr_input_pt2[:, -init_cols:]+1)), axis=1)
    
print('Day 15, part 2:', find_shortest_path(arr_input_pt2, (0,0), (arr_input_pt2.shape[0]-1,
                                                                   arr_input_pt2.shape[1]-1)))

Day 15, part 1: 2904
